# Preparação dos Documentos

In [5]:
import PyPDF2
import chromadb
import uuid
import os
import openai
import joblib
import numpy as np
import nest_asyncio  # noqa: E402
from dotenv import load_dotenv
from llama_parse import LlamaParse
from openai import OpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_unstructured import UnstructuredLoader
from langchain_chroma.vectorstores import Chroma
from getpass import getpass

nest_asyncio.apply()

In [8]:
CHUNK_SIZE = 2000
OFFSET = 200

In [9]:
openai_client = OpenAI(api_key= os.environ["OPENAI_API_KEY"])
# openai_client = OpenAI(api_key= openai_key)
# openai.api_key = os.getenv('OPENAI_API_KEY')

# Conexão com o cliente do banco de dados ChromaDB
chromadb_path = "G:/Drives compartilhados/RISCO E COMPLIANCE/Relatórios de Risco/Risco/FIDCS/SCRIPTS_RISCO/Projeto IA/Base ChromaDB/"
chroma_client = chromadb.PersistentClient(path= chromadb_path)
collection    = chroma_client.get_or_create_collection(name= "Regulamentos_FIDC_PLN_LLAMA")

In [ ]:
# Checa todo o conteúdo inserido no ChromaDB
collection.get()

In [11]:
def load_or_parse_data():

    data_file = "G:/Drives compartilhados/RISCO E COMPLIANCE/Relatórios de Risco/Risco/FIDCS/SCRIPTS_RISCO/Projeto IA/Base ChromaDB/llama/ICRED INSS - PARSED.pkl"

    if os.path.exists(data_file):
        # Load the parsed data from the file
        parsed_data = joblib.load(data_file)

    else:

        # Perform the parsing step and store the result in llama_parse_documents
        parsingInstructionFIDC = """The provided document is an investment fund regulation.
        The document may contain multiple attachments that should be considered as different segments.
        It includes many definitions, which may be contained within tables.
        Tables may be split into multiple pages, but should be considered as an unique table.
        """
        
        parser = LlamaParse(api_key= os.environ["LLAMA_API_KEY"],
                            result_type="markdown",
                            parsing_instruction= parsingInstructionFIDC,
                            page_separator = "\n== {pageNumber} ==\n",
                            #language = "pt",
                            max_timeout= 5000,)
        
        llama_parse_documents = parser.load_data("G:/Drives compartilhados/GESTAO/_Operacional/Planilhas Gestão/Scripts/temp/temp_regulamentos/ICRED INSS RESP LIMITADA FIDC 1.pdf")

        # Save the parsed data to a file
        print("Saving the parse results in .pkl format ..........")
        joblib.dump(llama_parse_documents, data_file)

        # Set the parsed data to the variable
        parsed_data = llama_parse_documents

    return parsed_data

In [12]:
# Chamada da API OpenAI para transformação do texto em vetor utilizando text-embedding
def get_embedding(text):

    embedding = OpenAIEmbeddings(
        model= "text-embedding-ada-002",
        chunk_size= CHUNK_SIZE
    )

    emb = embedding.embed_query(text)

    return emb

def create_vector_database():

    """
    Creates a vector database using document loaders and embeddings.

    This function loads urls, splits the loaded documents into chunks, 
    transforms them into embeddings using ada,
    and finally persists the embeddings into a Chroma vector database.

    """

    # Call the function to either load or parse the data
    llama_parse_documents = load_or_parse_data()
    print(llama_parse_documents[0].text[:300])

    # Open the file in append mode ('a')
    with open("G:/Drives compartilhados/RISCO E COMPLIANCE/Relatórios de Risco/Risco/FIDCS/SCRIPTS_RISCO/Projeto IA/Base ChromaDB/llama/output.md", 'a') as f:  
        for doc in llama_parse_documents:
            f.write(doc.text + '\n')

    markdown_path = "G:/Drives compartilhados/RISCO E COMPLIANCE/Relatórios de Risco/Risco/FIDCS/SCRIPTS_RISCO/Projeto IA/Base ChromaDB/llama/output.md"
    loader = UnstructuredLoader(markdown_path)

    #loader = DirectoryLoader('data/', glob="**/*.md", show_progress=True)
    documents = loader.load()
    
    # Split loaded documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size= CHUNK_SIZE, chunk_overlap=20)
    docs = text_splitter.split_documents(documents)

    #len(docs)
    print(f"length of documents loaded: {len(documents)}")
    print(f"total number of document chunks generated :{len(docs)}")
    #docs[0]

    # Initialize Embeddings
    embed_model = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")
    
    # Create and persist a Chroma vector database from the chunked documents
    vs = Chroma.from_documents(
        documents= docs,
        embedding= embed_model,
        persist_directory= "G:/Drives compartilhados/RISCO E COMPLIANCE/Relatórios de Risco/Risco/FIDCS/SCRIPTS_RISCO/Projeto IA/Base ChromaDB/",  # Local mode with in-memory storage only
        collection_name= "Regulamentos_FIDC_PLN_LLAMA"
    )

    #query it
    #query = "what is the agend of Financial Statements for 2022 ?"
    #found_doc = qdrant.similarity_search(query, k=3)
    #print(found_doc[0][:100])
    #print(qdrant.get())

    print('Vector DB created successfully !')
    return vs,embed_model

In [ ]:
vs,embed_model = create_vector_database()

In [ ]:
# Checa todo o conteúdo inserido no ChromaDB
collection.get()

# Implementação da RAG

In [ ]:
vectorstore = Chroma(embedding_function=embed_model,
                    persist_directory="chroma_db_llamaparse1",
                    collection_name="rag")

retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

# Aplicação do LLM

In [ ]:
question = """Fale sobre o MOVE BRASIL FIAGRO FIDC 1. Cite os participantes de sua estrutura 
(administrador, gestor, cedente, etc), discorra a respeito dos tipos
de direitos creditórios que podem ser adquiridos, quais são os eventos de avaliação do fundo
e como é paga a remuneração de gestora e administradora. Também inclua qualquer outro fator
relevante para a estrutura do fundo."""

In [ ]:
def set_custom_prompt(custom_prompt_template):
    """
    Prompt template for QA retrieval for each vectorstore
    """
    prompt = PromptTemplate(template= custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt

In [12]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

modelo = ChatOpenAI(model = "gpt-4o-mini", temperature= 0, max_tokens= 4096)

custom_prompt_template  = """Você é um assistente de IA que responde as dúvidas dos usuários com bases nos documentos abaixo.
Os documentos abaixo apresentam as fontes atualizadas e devem ser consideradas como verdade. Sempre que uma
pergunta oferecer o nome de um FIDC, você deve procurar pela melhor correspondência deste nome na lista de documentos.
Se uma informação não for encontrada nos documentos, diga que não foi encontrada. Não tente gerar qualquer definição.
Documentos: {documents}
""" #Caso o nome citado na pergunta não exista na lista de documentos, responda que ele não foi encontrado.

prompt = set_custom_prompt(custom_prompt_template)

messages=[
  SystemMessage(content= custom_prompt_template),
  HumanMessage(content= question)
]

prompt = set_custom_prompt()

########################### RESPONSE ###########################
PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of information to answer the user's question.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nContext: {context}\nQuestion: {question}\n\nOnly return the helpful answer below and nothing else.\nHelpful answer:\n")

In [ ]:
qa = RetrievalQA.from_chain_type(llm= modelo,
                               chain_type="stuff",
                               retriever=retriever,
                               return_source_documents=True,
                               chain_type_kwargs={"prompt": prompt})

In [13]:
answer = modelo.invoke(messages)

In [ ]:
answer.content